If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [ ]:
] activate ../ 

In [ ]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

In [ ]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [ ]:
using EmergencyBrakingSystem

pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = false

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 14.0
ped_v = 5.0/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -7.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -1.
ped_v = 0.0
ego_y = 0.0
# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.05
timestep_pomdp = 0.2 

pos_noise = 0.00
vel_noise = 0.00
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

emergency_braking_system = EmergencyBrakingSystem.EmergencySystem(a=LatLonAccel(0.0, 0.0),
        env=env,
        sensor=sensor, 
        obstacles=env.obstacles, 
        SAFETY_DISTANCE_LON=1.0,
        AX_MAX=-10.0,
        THRESHOLD_COLLISION_RATE = 0.6,
        THRESHOLD_TIME_TO_REACT = 0.99,    
        timestep=timestep)

pomdp.desired_velocity = ego_v
pomdp.ΔT = timestep
frenet_pedestrian_pomdp = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep_pomdp,
    pomdp=pomdp,
    policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)

pedestrian_system = PedestrianSystem(
    frenet_pedestrian_pomdp=frenet_pedestrian_pomdp, 
    emergency_braking_system=emergency_braking_system,
    update_tick_high_level_planner = timestep_pomdp / timestep,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v),
    timestep=timestep
)

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.0001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.0001) # dumb model



scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

pomdp.desired_velocity = ego_v
pomdp.ΔT = timestep
models[ego_id] = pedestrian_system

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.001) # dumb model



nticks = 200
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]
prediction = Vector{Array{Float64}}()
obs_callback = (ObservationPedestrianAvoidanceCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp, prediction),)

@elapsed begin
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
end

In [ ]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, prediction, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video
